# Improving Access to Clean, Safe Drinking Water in Tanzania
## Data Munging

Kenneth Liao

---

## Background

The UN publishes and reviews a list of least developed countries (LDC) every 3 years. LDCs are “low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.”$^{1}$. Tanzania has been classified as an LDC since the UN published the first list of LDCs in 1971$^{2}$. A common challenge of LDCs is a lack of infrastructure to support the development of the nation, including access to education and healthcare, waste management, and potable water.

According to UNICEF, as of 2017, more than 24 million Tanzanians lacked access to basic drinking water$^{3}$. This corresponds to only 56.7% of the country’s population having access to basic drinking water. Outside of developed urban areas, much of the potable water is accessed via water pumps. 

Taarifa is an open-source platform for crowd-sourced reporting and triaging of infrastructure related issues. Together with the Tanzanian Ministry of Water, data has been collected for thousands of water pumps throughout Tanzania. The goal of this project is to be able to predict the condition of these water pumps to improve maintenance, reduce pump downtime, and ensure basic water access for tens of millions of Tanzanians.

**References**

1. https://www.un.org/development/desa/dpad/least-developed-country-category.html
2. https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/publication/ldc_list.pdf
3. https://washwatch.org/en/countries/tanzania/summary/statistics/


### Problem Description

Predict the operating condition of water pumps in Tanzania given various metadata on each water pump.

### Strategy

The strategy will be to implement a Random Forest model for multiclass classification of the state of water pumps.

### Data

The dataset is provided by Taarifa, together with the Tanzanian Ministry of Water and is hosted by DrivenData.org:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

---

## Data Munging

In [24]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
from config import credentials
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import scipy.sparse
from scipy.sparse import csr_matrix
from sklearn.externals import joblib

init_notebook_mode(connected=True)

C:\Users\Kenny\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [2]:
# load the data
train = pd.read_csv('../data/train.csv')
train_labels = pd.read_csv('../data/train-labels.csv')

I'll start by removing the unwanted feature columns we identified in the EDA part of the analysis. This includes duplicate, irrelevant, and single value columns.

In [3]:
duplicated = ['recorded_by', 'payment_type', 'quality_group', 'quantity_group']

train_clean = train.drop(duplicated, axis=1)
train_clean.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'water_quality', 'quantity', 'source',
       'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [4]:
train_clean.set_index(['id', 'date_recorded'], inplace=True)

In [5]:
train_clean.head()

,,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,management,management_group,payment,water_quality,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,date_recorded,,,,,,,,,,,,,,,,,,,,,
69572,2011-03-14,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,...,vwc,user-group,pay annually,soft,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,2013-03-06,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,...,wug,user-group,never pay,soft,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,2013-02-25,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,...,vwc,user-group,pay per bucket,soft,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,2013-01-28,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,...,vwc,user-group,never pay,soft,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,2011-07-13,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,...,other,other,never pay,soft,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


Next, I need to convert the categorical text features into dummy variables.

In [6]:
# list of all categorical variables
cat_cols = []
for col in train_clean.columns:
    if train_clean[col].dtype == 'object':
        cat_cols.append(col)
cat_cols

['funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'water_quality',
 'quantity',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [7]:
%%time
cat_dummies = pd.get_dummies(train_clean[cat_cols], dummy_na=True)

Wall time: 38.2 s


I use `pd.get_dummies` with the argument dummy_na=True so that null values are not ignored. They are instead encoded the same as all other values so each feature will have a null dummy variable, indicated whether the sample was null or not for that feature. The resulting categorical feature set now has 65,828 features.

In [8]:
cat_dummies.head()

,,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,funder_Abasia,funder_Abc-ihushi Development Cent,funder_Abd,funder_Abdala,funder_Abddwe,funder_Abdul,...,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_nan,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,waterpoint_type_group_nan
id,date_recorded,,,,,,,,,,,,,,,,,,,,,
69572,2011-03-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8776,2013-03-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
34310,2013-02-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
67743,2013-01-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
19728,2011-07-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [9]:
# list of all numerical variables
num_cols = []
for col in train_clean.columns:
    if train_clean[col].dtype != 'object':
        num_cols.append(col)
num_cols

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [10]:
numerical = train_clean[num_cols]
numerical.head()

,,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,date_recorded,,,,,,,,,
69572,2011-03-14,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
8776,2013-03-06,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
34310,2013-02-25,25.0,686,37.460664,-3.821329,0,21,4,250,2009
67743,2013-01-28,0.0,263,38.486161,-11.155298,0,90,63,58,1986
19728,2011-07-13,0.0,0,31.130847,-1.825359,0,18,1,0,0


The dataframe above contains the numerical type columns. Although tree-based methods do not require feature scaling, I'd like to compare the performance of the random forest model with a logistic regression model. Logistic regression models perform optimally on scaled feature data. I'll use sklearn's `StandardScaler` to get the standard z score. I'll thus have two versions of the training data, one with numerical features scaled and the other without scaling.

In [11]:
# define scaler (subtracts mean and divides by std dev)
scaler = StandardScaler()

# fit the data nd replace dataframe
numerical_scaled = pd.DataFrame(scaler.fit_transform(numerical), index=numerical.index, columns=num_cols)

In [12]:
numerical.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [13]:
numerical_scaled.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04,5.940000e+04
mean,5.364466e-16,1.731275e-16,-9.878798e-16,1.358238e-16,-5.503924e-16,-2.557812e-16,1.213732e-16,-1.367457e-16,3.053039e-16
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-1.059700e-01,-1.094050e+00,-5.188895e+00,-2.017454e+00,-3.874931e-02,-8.129183e-01,-5.843886e-01,-3.815871e-01,-1.366788e+00
25%,-1.059700e-01,-9.642001e-01,-1.503004e-01,-9.621840e-01,-3.874931e-02,-5.854809e-01,-3.767812e-01,-3.815871e-01,-1.366788e+00
50%,-1.059700e-01,-4.318175e-01,1.265828e-01,2.323277e-01,-3.874931e-02,-1.874655e-01,-2.729775e-01,-3.285623e-01,7.201960e-01
75%,-9.929791e-02,9.391760e-01,4.721763e-01,8.078349e-01,-3.874931e-02,9.683127e-02,-6.537012e-02,7.442553e-02,7.391112e-01
max,1.166561e+02,3.032277e+00,9.543790e-01,1.936878e+00,1.451052e+02,4.759298e+00,7.719908e+00,6.430857e+01,7.485689e-01


Using the `describe` method above, we can see that the data has been scaled properly with a mean of 0 for each column and a standard deviation of 1.

Luckily, none of the numerical columns have null values. 

In [15]:
# merge data back together.
train_full = pd.concat([cat_dummies, numerical], axis=1)
train_full_scaled = pd.concat([cat_dummies, numerical_scaled], axis=1)
train_full.head()

,,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,funder_Abasia,funder_Abc-ihushi Development Cent,funder_Abd,funder_Abdala,funder_Abddwe,funder_Abdul,...,waterpoint_type_group_nan,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,date_recorded,,,,,,,,,,,,,,,,,,,,,
69572,2011-03-14,0,0,0,0,0,0,0,0,0,0,...,0,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
8776,2013-03-06,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
34310,2013-02-25,0,0,0,0,0,0,0,0,0,0,...,0,25.0,686,37.460664,-3.821329,0,21,4,250,2009
67743,2013-01-28,0,0,0,0,0,0,0,0,0,0,...,0,0.0,263,38.486161,-11.155298,0,90,63,58,1986
19728,2011-07-13,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,31.130847,-1.825359,0,18,1,0,0


The last thing to do here is to convert the `date_recorded` into a dummy variable for year, month, and day. I'm also introducing one new feature which is the number of days between the date recorded and construction year. This will give the model an idea of how long the pump was in service.

In [16]:
train_full = train_full.reset_index().set_index('id')
train_full['year_recorded'] = pd.to_datetime(train_full.date_recorded).dt.year
train_full['month_recorded'] = pd.to_datetime(train_full.date_recorded).dt.month
train_full['day_recorded'] = pd.to_datetime(train_full.date_recorded).dt.day
train_full = train_full.drop('date_recorded', axis=1)
train_full['years_since_install'] = train_full['year_recorded'] - train_full['construction_year']

In [17]:
train_full_scaled = train_full_scaled.reset_index().set_index('id')
train_full_scaled['year_recorded'] = pd.to_datetime(train_full_scaled.date_recorded).dt.year
train_full_scaled['month_recorded'] = pd.to_datetime(train_full_scaled.date_recorded).dt.month
train_full_scaled['day_recorded'] = pd.to_datetime(train_full_scaled.date_recorded).dt.day
train_full_scaled = train_full_scaled.drop('date_recorded', axis=1)
train_full_scaled['years_since_install'] = train_full_scaled['year_recorded'] - train_full_scaled['construction_year']

The class labels will also be converted to a numerical data type using the mapping below.

In [18]:
train_labels = train_labels.set_index('id')
train_labels = train_labels['status_group'].map({'functional': 0, 'functional needs repair': 1, 'non functional': 2})

The model will return a value error if a feature name contains the values [, ], or <. Therefore, I'll use regex to remove those characters from column names.

In [19]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
train_full.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in train_full.columns.values]

train_full_scaled.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in train_full_scaled.columns.values]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train_full, train_labels, test_size=0.33, random_state=42)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(train_full_scaled, train_labels, test_size=0.33, random_state=42)

For optimizing the model implementation, I'm going to convert the dense matrices `X_train` and `X_test` into sparse matrices. Since the majority of the training data are 0s and 1s, the training will be much more efficient if the data is saved as a sparse matrix because it will take much less space in memory.

In [21]:
# convert dense matrices to sparse matrices
X_train_s = csr_matrix(X_train)
X_test_s = csr_matrix(X_test)
X_train_s_scaled = csr_matrix(X_train_scaled)
X_test_s_scaled = csr_matrix(X_test_scaled)

Finally, I'll export all of the data to files.

In [22]:
X_train.to_pickle('../data/X_train.pkl')
X_test.to_pickle('../data/X_test.pkl')
X_train_scaled.to_pickle('../data/X_train_scaled.pkl')
X_test_scaled.to_pickle('../data/X_test_scaled.pkl')

# save sparse versions of the matrices for training on
scipy.sparse.save_npz('../data/X_train_s.npz', X_train_s)
scipy.sparse.save_npz('../data/X_test_s.npz', X_test_s)
scipy.sparse.save_npz('../data/X_train_s_scaled.npz', X_train_s_scaled)
scipy.sparse.save_npz('../data/X_test_s_scaled.npz', X_test_s_scaled)

y_train.to_pickle('../data/y_train.pkl')
y_test.to_pickle('../data/y_test.pkl')

In [25]:
# save the feature names
feature_names = X_train.columns

joblib.dump(feature_names, '../data/feature_names.pkl')

['feature_names.pkl']

In [21]:
X_train.head()

,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,funder_Abasia,funder_Abc-ihushi Development Cent,funder_Abd,funder_Abdala,funder_Abddwe,funder_Abdul,...,latitude,num_private,region_code,district_code,population,construction_year,year_recorded,month_recorded,day_recorded,years_since_install
id,,,,,,,,,,,,,,,,,,,,,
60371,0,0,0,0,0,0,0,0,0,0,...,0.910024,-0.038749,0.153691,0.246041,-0.381587,-1.366788,2011,7,21,2012.366788
17088,0,0,0,0,0,0,0,0,0,0,...,-0.110139,-0.038749,-0.812918,-0.272978,-0.381587,-1.366788,2011,3,11,2012.366788
16532,0,0,0,0,0,0,0,0,0,0,...,1.362428,-0.038749,0.153691,-0.480585,-0.381587,-1.366788,2011,7,18,2012.366788
11098,0,0,0,0,0,0,0,0,0,0,...,0.855770,-0.038749,-0.699200,-0.480585,-0.379466,0.708637,2013,2,20,2012.291363
20249,0,0,0,0,0,0,0,0,0,0,...,0.437501,-0.038749,-0.073747,-0.480585,-0.381587,-1.366788,2013,1,18,2014.366788


In [22]:
y_train.head()

id
60371    2
17088    0
16532    2
11098    0
20249    0
Name: status_group, dtype: int64

The full dataset is now ready to train on. There may be issues with the dimension of this dataset after converting to dummy variables. The shape of the dataset is now 59400 X 65834. If the model shows poor performance, it may benefit by using another model to reduce the number of features to those which are most important. This can be done with a number of techniques including PCA, step-wise feature selection, and genetic algorithms for feature selection.